# FBA Quant FE Session HW7

## Woohyuk Choi


In [2]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

### Problem 1

In [4]:
df = pd.read_excel('EquityDerivsPractice_PSet3.xlsx', sheet_name = 1, index_col = 0)
log_return = np.log(df.div(df.shift(1))).dropna()

days = 252
annualized_return = log_return.mean(axis=0) * days
annualized_vol = log_return.std(axis=0) * np.sqrt(days)


summary_df = pd.DataFrame({"annualized_return":annualized_return, "annualized_vol":
              annualized_vol})
summary_df

,annualized_return,annualized_vol
Stock Path # 1,-0.183846,0.285351
Stock Path # 2,0.439480,0.340784
Stock Path # 3,1.247081,0.578860
Stock Path # 4,-0.760258,0.450566


### Problem 2 - (a)

The Black-Scholes formula is a mathematical model used to calculate the theoretical price of an option.

The Black-Scholes formulas for call and put options are as follows:

For a call option:
$$ C = SN(d_1) - Xe^{-rt} N(d_2)$$

For a put option:
$$P = X e^{-rt} N(-d_2) - S N(-d_1)$$

d1 and d2 are variables calculated as:
$d_1 = \frac {(\ln(\frac S X) + (r + \frac {\sigma^2} 2) t )}  {\sigma \sqrt t}$ , $d_2 = d_1 - \sigma \sqrt t$

Let's substitute your data into these formulas:

S = 50 (stock price)
X = 50 (exercise price)
r = 0.03 (annual risk-free interest rate)
t = 0.5 (6 months is half a year)
σ = 0.5 (standard deviation)

Now let's compute $d_1$ and $d_2$:

$$d_1 = (ln(\frac {50} {50}) + (0.03 + \frac {0.5^2} 2) 0.5 ) / (0.5 \sqrt{0.5}) = 0.15$$
$$d_2 = d_1 - 0.5\sqrt{0.5} = -0.2$$

Let's substitute d1, d2, and all the other values into the Black-Scholes formulas:

$$N(d_1) = 0.5596$$
$$N(d_2) = 0.4207$$
$$N(-d_1) = 1 - N(d_1) = 0.4404$$
$$N(-d_2) = 1 - N(d_2) = 0.5793$$

Now, we substitute these values into the formulas for the call and put options:

$$ C = 50 \times 0.5596 - 50 \times e^{-0.03*0.5} \times 0.4207 = 27.98$$

$$ P = 50 \times e^{-0.03*0.5} \times 0.5793 - 50 \times 0.4404 = 4.39$$ 


### Problem 2 - (b)

**Underlying Asset Price (S)**: As the current price of the underlying asset increases, the value of the call option increases. This is because the option gives you the right to buy the asset at the strike price. If the current price is higher, you can buy at a lower price and sell at the current price, thus making a profit. The rate of change of the call price with respect to the asset price is represented by Delta.

**Exercise Price (X)**: If the exercise price increases while keeping all other parameters constant, the value of a call option will decrease. The reason is that the call option gives you the right to buy the underlying asset at the strike price. If the strike price is higher, the option becomes less valuable because you have the right to buy at a higher price.

**Time to Expiration (t)**: Generally, the longer the time to expiration, the more valuable the option becomes. This is because there is more time for the price of the underlying asset to move favorably (i.e., above the strike price for a call option). The rate of change of the call price with respect to time to expiration is represented by Theta.

**Standard Deviation/Volatility (σ)**: If the volatility of the underlying asset increases, the price of the call option will increase. Higher volatility implies that the underlying asset has a higher chance to move significantly in price, increasing the chance it will be above the strike price at expiration. The rate of change of the call price with respect to volatility is represented by Vega.

**Risk-free Interest Rate (r)**: An increase in the risk-free interest rate increases the price of a call option. This is because the cost of carrying the position (the opportunity cost of money) is reduced by higher interest rates. This is represented by Rho.

### Not My Idea

### Problem 3 - (a)

For call option in Black-Scholes Model, delta of the option is

$$ \Delta = N(d_1) $$

Where $$d_1 = \frac {\ln (\frac S K) + (r+ \frac {\sigma^2} 2) T} {\sigma \sqrt{T}}$$

$$ \Gamma = \frac {d \Delta} {dS} = N'(d_1) \frac {\partial d_1} {\partial S} = N'(d_1) \frac 1 {\partial \sqrt {T}} \times \frac K S \times \frac 1 K = N'(d_1) \times \frac 1 {S \sigma \sqrt T}$$ 

For put option in Black-Scholes model, delta of the option is 
$$\Delta = N(d_1) - 1$$

So it has the same $\Gamma$ as call option with same parameters.

### Problem 3 - (b)

$$\Theta + rs\Delta + \frac 1 2 \sigma^2 S^2 \Gamma = rP$$
$$\Theta = rP - rs\Delta - \frac 1 2 \sigma^2 S^2 \Gamma$$

We know that 
$$P = S N(d_1) - K e^{-rT} N(d_2)$$
$$\Delta = N(d_1)$$
$$\Gamma = N'(d_1) \times \frac 1 {S \sigma \sqrt T}$$ 

So, 
$$\Theta = r(-Ke^{-rT}N(d_2)) - \frac 1 2 \sigma^2 S^2 (N'(d_1) \times \frac 1 {S\sigma \sqrt T})$$
$$ = - \frac {S N'(d_1) \sigma} {2 \sqrt T} - r K e^{-rT} N(d_2)$$

### Problem 4

In [3]:
def calculate_option_price(S, K, r, sigma, T):
  d1 = (np.log(S/K) + (r+sigma**2/2) * T) / (sigma * np.sqrt(T))
  d2 = (np.log(S/K) + (r-sigma**2/2) * T) / (sigma * np.sqrt(T))
  rv = norm(0,1)
  
  call = S*rv.cdf(d1) - K*np.exp(-r*T)*rv.cdf(d2)
  put = K*np.exp(-r*T)*rv.cdf(-d2) - S*rv.cdf(-d1)
  return call, put

In [9]:
S = 50 
K = 50 
r = 0.03 
sigma = 0.3 
T = 0.5 

call, put = calculate_option_price(S, K, r, sigma, T)
print(f"Price of Call Option: {call:.4f} (Constant Volatility)")

Price of Call Option: 4.5747 (Constant Volatility)


In [1]:
import numpy as np
from scipy.stats import norm

# Parameters
S = 50  # Spot price
K = 50  # Strike price
r = 0.03  # Risk-free interest rate
T = 0.5 # Time to expiration (in years)
N = 10000  # Number of simulations

# Constant volatility
sigma_const = 0.30

# Random volatility (normal distribution)
mu = 0.30  # Mean
sigma_dist = 0.05  # Standard deviation

# Black-Scholes formula for a European call option
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = (S * norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * norm.cdf(d2, 0.0, 1.0))
    return call_price

# Price option with constant volatility
call_price_const = black_scholes_call(S, K, T, r, sigma_const)
print(f'Call price with constant volatility: {call_price_const}')

# Price option with random volatility
random_volatilities = np.random.normal(mu, sigma_dist, N)
call_prices_random = [black_scholes_call(S, K, T, r, sigma) for sigma in random_volatilities]
call_price_random_mean = np.mean(call_prices_random)
print(f'Mean call price with random volatility: {call_price_random_mean}')


Call price with constant volatility: 4.57469928882983
Mean call price with random volatility: 4.582072100297211
